In [2]:
from utilities.utils import get_fulldata, process_land_area

usecols = ["name_of_ryot", "caste", "6", "7", "8"]
df = get_fulldata(**{"usecols": usecols, "low_memory": False}).pipe(process_land_area)
print(len(df))
df.head()

/home/lsys/land/venv_land/lib/python3.10/site-packages/pandas_flavor/register.py:161: FutureWarning: This function will be deprecated in a 1.x release. Please use `pd.DataFrame.rename` instead.
  return method(self._obj, *args, **kwargs)


38589908


,name_of_ryot,caste,acres,decimals,hectare,tt_area_acre
0,कपिलदेव पाठक,ब्रहामण,0.0,66.0,0.0,0.66
1,कपिलदेव पाठक,ब्रहामण,0.0,8.0,0.0,0.08
2,कपिलदेव पाठक,ब्रहामण,0.0,6.0,0.0,0.06
3,कपिलदेव पाठक,ब्रहामण,0.0,3.0,0.0,0.03
4,कपिलदेव पाठक,ब्रहामण,0.0,36.0,0.0,0.36


In [21]:
df_ryot_caste = (
    df.dropna(subset="name_of_ryot")
    .select_columns(["name_of_ryot", "caste"])
    .drop_duplicates(subset=["name_of_ryot"])
    .reset_index(drop=True)
)
# df_ryot_caste.to_csv("../data/ryot_hindi_caste.csv.gz", index=False, compression="gzip")
print(len(df_ryot_caste))
df_ryot_caste

3197303


,name_of_ryot,caste
0,कपिलदेव पाठक,ब्रहामण
1,किशोरी राय,यादव
2,किताब महतो,कुर्मी
3,जगदेव राय,यादव
4,अकलु राय,यादव
...,...,...
3197298,चरीत्तर चौधरी,कलवार
3197299,विहार ग्राम उधोग संघ सरैया,NaN
3197300,बिहार सरकार कृषी विभाग वीजगुणा,NaN
3197301,मु. मंचनिया,कोइरी


In [16]:
problematic_pattern = r'[°\[\]\(\)\\\/;:&"\'\+\-०-९⁪a-zA-Z0-9]'
matches = df_ryot_caste[df_ryot_caste["name_of_ryot"].str.contains(problematic_pattern, na=False, regex=True)]
print(f"Found {len(matches)} rows with problematic characters")
print(matches["name_of_ryot"].sample(n=100).tolist())

Found 282600 rows with problematic characters
['मो0 सकिया', 'म0 साहीद', 'श्री श्री108 रामजानकी सेवा', 'मो0 कतामो उद्दीन', 'विमलेश वि0', 'पूष्परंजन कु0 सिंह', 'शो0 लवदीन', 'शिव वह्न वगै0', 'मो0 अकबर आलम', 'मु0 जसरीना', 'चन्दन चौ0', 'मु0 हरकेसीया', 'सजलकु0', 'मो0 शदाब अहमद', 'मु0 घरनी', 'सुवेस वि0', 'शे0 हिफायत', 'अ0  दुलवारी', 'रामदेव सिंह (गैर0 मालिक)', 'शे0 तसीरुदीन', 'बेचु कपरी ईत्यादि व.स.न. 361', 'मु0 भगजोगनी', 'बी0 वी0 कलीमन', 'अजीज अंसारी (गैर0 आम)', 'मो0 मुजतवा खान', 'स्\u200dव0 रामसुमिरण सिंह', 'स्\u200dव0 धनपत महतो', 'मु0 मतारी', 'जग ना0 राय', 'पंकज व  निरंजन कु0 सिंह', 'अजय कु0 मिश्र', 'मो०हैदरअली', 'मु0 जान बेबी', 'तेजना0 सिंह', 'मु0 भुदीया', "तारावती देवी '", 'BHAGO DEVI', 'रामाकांत चौबे वगै0', 'स्\u200dव0 कृपाल यादव', 'मसो0 शहजादी', 'महावीर राम वो0', 'मु0 रुपा कुंवर', 'मो0 मोजीबुर्रहमान', 'कामेश्वर प्रसाद- यादव', 'अबूल ई0', 'मो0 समनुर', 'बाबुराजेन्दर प्र0', 'मो0 जुबवेर आल्म', 'वकटेश शर्मा वो0 बिगन शर्मा', 'मु0 रईश उद्दीन', 'धनश्\u200dयाम सिहं वगै0', 'श्रीश्री 108 श्री श्री

In [25]:
def clean_hindi_names(df, name_column='name_of_ryot'):
    r"""
    Clean Hindi names by removing problematic characters.
    
    Removes (replaces with ''):
    - Special characters: °[]()\/;:&"'+-
    - English letters and numbers: a-zA-Z0-9
    - Devanagari digits: ०-९
    - Special unicode characters
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    name_column : str
        Name of the column containing names to clean
        
    Returns:
    --------
    pd.DataFrame
        Dataframe with cleaned names
    """
    problematic_pattern = r'[°\[\]\(\)\\\/;:&"\'\+\-०-९⁪a-zA-Z0-9]'
    
    df[name_column] = df[name_column].str.replace(problematic_pattern, '', regex=True)
    
    df[name_column] = df[name_column].str.replace(r'\s+', ' ', regex=True)
    
    df[name_column] = df[name_column].str.strip()
    
    return df

In [26]:
df_ryot_caste = clean_hindi_names(df_ryot_caste)
# df_ryot_caste.to_csv("../data/ryot_hindi_caste.csv.gz", index=False, compression="gzip")
df_ryot_caste

,name_of_ryot,caste
0,कपिलदेव पाठक,ब्रहामण
1,किशोरी राय,यादव
2,किताब महतो,कुर्मी
3,जगदेव राय,यादव
4,अकलु राय,यादव
...,...,...
3197298,चरीत्तर चौधरी,कलवार
3197299,विहार ग्राम उधोग संघ सरैया,NaN
3197300,बिहार सरकार कृषी विभाग वीजगुणा,NaN
3197301,मु. मंचनिया,कोइरी


In [32]:
matches = df_ryot_caste[df_ryot_caste["name_of_ryot"].str.contains(problematic_pattern, na=False, regex=True)]
print(f"Found {len(matches)} rows with problematic characters")
try:
    print(matches["name_of_ryot"].sample(n=100).tolist())
except ValueError:
    pass

Found 0 rows with problematic characters


In [28]:
df_ryot_caste[df_ryot_caste['name_of_ryot'].str.len() == 0]

,name_of_ryot,caste
6142,,NaN
21363,,NaN
54487,,राजपूत
83984,,मुशलमान
83991,,शेख
...,...,...
3186895,,यादव
3187514,,यादव
3188367,,यादव
3188384,,कुर्मी


In [33]:
df_ryot_caste = df_ryot_caste[df_ryot_caste['name_of_ryot'].str.len() > 0]
df_ryot_caste.to_csv("../data/ryot_hindi_caste.csv.gz", index=False, compression="gzip")
df_ryot_caste

,name_of_ryot,caste
0,कपिलदेव पाठक,ब्रहामण
1,किशोरी राय,यादव
2,किताब महतो,कुर्मी
3,जगदेव राय,यादव
4,अकलु राय,यादव
...,...,...
3197298,चरीत्तर चौधरी,कलवार
3197299,विहार ग्राम उधोग संघ सरैया,NaN
3197300,बिहार सरकार कृषी विभाग वीजगुणा,NaN
3197301,मु. मंचनिया,कोइरी
